In [2]:
import torch
import tensorflow as tf
import pandas as pd
import numpy as np

In [305]:
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
outputs= np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

inputs=torch.from_numpy(inputs)
outputs=torch.from_numpy(outputs)

In [308]:
print(inputs.shape)
print(outputs.shape)

torch.Size([5, 3])
torch.Size([5, 2])


In [411]:
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.1549, -0.5516,  0.6967],
        [-1.2601,  0.1069,  1.7034]], requires_grad=True)
tensor([ 0.5592, -0.2986], requires_grad=True)


In [311]:
# Matrix multiplication
torch.tensor([2.,3,4]) @ w.t() + b

tensor([2.0432, 5.6977], grad_fn=<AddBackward0>)

In [312]:
def model(x):
    #y = x*w^t +b
    return x @ w.t() +b

In [412]:
preds = model(inputs)
print(preds)

tensor([[-17.7506, -11.8794],
        [-17.4921,   3.4546],
        [-46.4281,   3.1912],
        [-13.1849, -61.2086],
        [-14.3161,  42.2528]], grad_fn=<AddBackward0>)


In [413]:
print(outputs)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [414]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [415]:
loss=mse(preds,outputs)
loss

tensor(10611.2070, grad_fn=<DivBackward0>)

In [416]:
loss.backward()

In [417]:
print(w)
print(w.grad)

tensor([[-0.1549, -0.5516,  0.6967],
        [-1.2601,  0.1069,  1.7034]], requires_grad=True)
tensor([[-8084.4985, -9710.2520, -5716.7139],
        [-8292.0059, -8610.9990, -5259.7305]])


In [418]:
print(b)
print(b.grad)

tensor([ 0.5592, -0.2986], requires_grad=True)
tensor([-98.0344, -96.8379])


In [419]:
#Reset gradient
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [420]:
preds = model(inputs)
loss=mse(preds,outputs)
loss.backward()
print(w)
print(b)
print("dy/dw ",w.grad)
print("dy/db ",b.grad)

tensor([[-0.1549, -0.5516,  0.6967],
        [-1.2601,  0.1069,  1.7034]], requires_grad=True)
tensor([ 0.5592, -0.2986], requires_grad=True)
dy/dw  tensor([[-8084.4985, -9710.2520, -5716.7139],
        [-8292.0059, -8610.9990, -5259.7305]])
dy/db  tensor([-98.0344, -96.8379])


In [421]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [422]:
print(w)
print(b)

tensor([[-0.0741, -0.4545,  0.7539],
        [-1.1772,  0.1930,  1.7560]], requires_grad=True)
tensor([ 0.5602, -0.2976], requires_grad=True)


In [423]:
preds = model(inputs)
loss = mse(preds, outputs)
print(loss)

tensor(7306.5273, grad_fn=<DivBackward0>)


In [424]:
# Train for 100 epochs
for i in range(10):
    preds = model(inputs)
    loss = mse(preds, outputs)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()

In [425]:
preds = model(inputs)
loss = mse(preds, outputs)
print(loss)

tensor(569.7829, grad_fn=<DivBackward0>)


In [427]:
preds

tensor([[ 55.4778,  57.8956],
        [ 79.0118,  94.9498],
        [ 70.2281, 110.8220],
        [ 56.9107,   9.1584],
        [ 79.7971, 129.3748]], grad_fn=<AddBackward0>)

In [428]:
import torch.nn as nn

In [429]:
inputs = np.array([[73, 67, 43], [91, 88, 64], [87, 134, 58], 
                   [102, 43, 37], [69, 96, 70], [73, 67, 43], 
                   [91, 88, 64], [87, 134, 58], [102, 43, 37], 
                   [69, 96, 70], [73, 67, 43], [91, 88, 64], 
                   [87, 134, 58], [102, 43, 37], [69, 96, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], [81, 101], [119, 133], 
                    [22, 37], [103, 119], [56, 70], 
                    [81, 101], [119, 133], [22, 37], 
                    [103, 119], [56, 70], [81, 101], 
                    [119, 133], [22, 37], [103, 119]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [431]:
print(inputs.shape)
print(targets.shape)

torch.Size([15, 3])
torch.Size([15, 2])


In [432]:
from torch.utils.data import TensorDataset

In [433]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [434]:
from torch.utils.data import DataLoader

In [436]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [437]:
train_dl

In [442]:
for xb, yb in train_dl:
    print(xb)
    print(yb)
    break

tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 73.,  67.,  43.],
        [102.,  43.,  37.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [ 56.,  70.],
        [ 56.,  70.],
        [ 22.,  37.]])


In [448]:
model=nn.Linear(in_features=3,out_features=2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[-0.0648,  0.0127,  0.1507],
        [-0.5539, -0.3884, -0.1451]], requires_grad=True)
Parameter containing:
tensor([ 0.0411, -0.1447], requires_grad=True)


In [453]:
list(model.parameters())

[Parameter containing:
 tensor([[-0.0648,  0.0127,  0.1507],
         [-0.5539, -0.3884, -0.1451]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0411, -0.1447], requires_grad=True)]

In [456]:
preds=model(inputs)
preds

tensor([[   2.6385,  -72.8474],
        [   4.9024,  -94.0228],
        [   4.8405, -108.8041],
        [  -0.4494,  -78.7189],
        [   7.3339,  -85.8140],
        [   2.6385,  -72.8474],
        [   4.9024,  -94.0228],
        [   4.8405, -108.8041],
        [  -0.4494,  -78.7189],
        [   7.3339,  -85.8140],
        [   2.6385,  -72.8474],
        [   4.9024,  -94.0228],
        [   4.8405, -108.8041],
        [  -0.4494,  -78.7189],
        [   7.3339,  -85.8140]], grad_fn=<AddmmBackward>)

In [459]:
import torch.nn.functional as F

In [460]:
loss_func=F.mse_loss

In [462]:
loss=loss_func(preds,targets)
loss

tensor(20357.4766, grad_fn=<MseLossBackward>)

In [464]:
#Creating optimizer
opt=torch.optim.SGD(model.parameters(),lr=1e-5)

In [468]:
def fit(num_epochs, model, loss_fn, opt, train_dl):
    for epoch in range(num_epochs):
        
        for xb,yb in train_dl:
            
            pred = model(xb)
            loss = loss_fn(pred, yb)
        
            loss.backward()
            
            #Update params
            opt.step()
            
            opt.zero_grad()
        
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [469]:
fit(100,model,loss_func,opt,train_dl)

Epoch [10/100], Loss: 17.0282
Epoch [20/100], Loss: 7.5558
Epoch [30/100], Loss: 12.7762
Epoch [40/100], Loss: 13.1480
Epoch [50/100], Loss: 4.1602
Epoch [60/100], Loss: 10.8704
Epoch [70/100], Loss: 4.3989
Epoch [80/100], Loss: 2.3724
Epoch [90/100], Loss: 3.3661
Epoch [100/100], Loss: 2.9720


In [471]:
preds = model(inputs)
preds

tensor([[ 57.3838,  70.5564],
        [ 81.2327,  98.8267],
        [120.2526, 136.3528],
        [ 22.4414,  38.7018],
        [ 99.4621, 114.9484],
        [ 57.3838,  70.5564],
        [ 81.2327,  98.8267],
        [120.2526, 136.3528],
        [ 22.4414,  38.7018],
        [ 99.4621, 114.9484],
        [ 57.3838,  70.5564],
        [ 81.2327,  98.8267],
        [120.2526, 136.3528],
        [ 22.4414,  38.7018],
        [ 99.4621, 114.9484]], grad_fn=<AddmmBackward>)

In [472]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

In [517]:
targets-preds

tensor([[-1.3838, -0.5564],
        [-0.2327,  2.1733],
        [-1.2526, -3.3528],
        [-0.4414, -1.7018],
        [ 3.5379,  4.0516],
        [-1.3838, -0.5564],
        [-0.2327,  2.1733],
        [-1.2526, -3.3528],
        [-0.4414, -1.7018],
        [ 3.5379,  4.0516],
        [-1.3838, -0.5564],
        [-0.2327,  2.1733],
        [-1.2526, -3.3528],
        [-0.4414, -1.7018],
        [ 3.5379,  4.0516]], grad_fn=<SubBackward0>)